#### Ejercicio 7

Escribir un código para estimar la norma infinito de una matriz, usar la fórmula cerrada. Cómo
serı́a un código si no tenemos una fórmula cerrada? Comparar.

##### Solución

Partiendo de la definición de norma infinito para una matriz a partir de la norma vectorial:

$$
\left\| A \right\|_{\infty} = \sup \left\{ \left\| Ax \right\|_{\infty} : \left\| x \right\|_{\infty} = 1 \right\}
$$

Y aplicando la fórmula cerrada para una matriz 
cuadrada de dimensión $n$ como:

$$
\left\| A \right\|_{\infty} = \max_{1 \le i \le n} \sum_{j=i}^n \left| a_{ij} \right|
$$

Podemos implementarla con la siguiente función `infnorm()`:

In [1]:
def infnorm(matrix):
    return max(sum(abs(elem) for elem in row) for row in matrix)

La verificamos para la matriz ejemplo de las notas de 
clase:

$$
\left( \begin{matrix} 1 & 1 \\ 3 & 0 \end{matrix} \right)
$$

En nuestro caso:

In [2]:
infnorm([[1,1],[3,0]])

3

En el caso de que no fuera posible obtener
la fórmula cerrada de nuestra norma (como sería
para el caso de una norma $p$ en general)
deberíamos aplicar un procedimiento probabilístico
generando la mayor cantidad posible de vectores $x$
que posean norma 1. Definimo así una segunda versión
de nuestra función:

In [3]:
from random import randint
from numpy import matmul
def infnorm2(matrix, sample_size):
    def infnormvec(vector):
        return max(abs(elem) for elem in vector)
    def vecnorm(vector):
        return [elem / infnormvec(vector) for elem in vector]
    vectors = ([randint(-100, 100) for _ in range(len(matrix[0]))] for _ in range(sample_size))
    return max(infnormvec(matmul(matrix, vecnorm(vector))) for vector in vectors)

Para determinadas matrices se puede obtener un resultado
aceptable con pocas muestras. Reutilizando nuestra matriz
de ejemplo:

In [4]:
infnorm2([[1,1],[3,0]], 10)

3.0

Sin embargo, otras matrices nos obligan a una mayor 
cantidad de muestra para objener un resultado 
cercano al de la fórmula cerrada. Con ayuda de nuestra
función auxiliar `table` que se lista en el Anexo, 
construiremos una tabla comparativa para los 
resultados obtenidos con una matriz en el cálculo
de su norma infinita con diferentes tamaños de 
muestra aleatorias de vectores de norma infinita 1:

In [5]:
from auxiliares import tabla
tabla(
    [10 ** x for x in range(6)],
    ["Tamaño de muestra", "$\\begin{Vmatrix}A\end{Vmatrix}_{\infty}$"],
    [repr, lambda x: infnorm2([[1,-2,5],[1,2,4]], x)]  
)

|Tamaño de muestra|$\begin{Vmatrix}A\end{Vmatrix}_{\infty}$|
|:-:|:-:
|1|1.9878048780487805|
|10|7.46969696969697|
|100|7.703703703703704|
|1000|7.786516853932584|
|10000|7.9714285714285715|
|100000|8.0|


#### Ejercicio 11

Escribir un código para resolver el sistema matricial $Ax = b$ donde $A$ es una matriz que es una
permutación de una matríz triangular superior.

##### Solución

Asumiendo que la matríz $A$ es una permutación de una matriz triangular
superior, aplicamos el algoritmo de resolución de una tal matriz para 
un vector de valores independientes, siguiendo un ordenamiento dado 
por un vector de permutaciones.
En caso que dicho vector de permutaciones no sea proveído, lo calculamos.
Dada la similitud de los algoritmos, agregamos un parámetro que indica
si la matriz es triangular superior o inferior. Así, la implementación 
de la función `solvetrian` resulta:

In [6]:
from itertools import takewhile
from functools import reduce


def solvetrian(matrix, values, perm=None, lower=False):
    n = len(matrix)
    ind = range(n)
    if not perm:
        keys = [
            reduce(
                lambda s, i: s + 1,
                takewhile(lambda x: x == 0 ^ lower, row),
                0,
            )
            for row in matrix
        ]
        perm = sorted(ind, key=(lambda x: keys[x]))
    return reduce(
        lambda r, i: [
            (
                values[perm[i]]
                - matmul(
                    matrix[perm[i]][
                        slice(i) if lower else slice(i + 1, n)
                    ],
                    r,
                )
            )
            / matrix[perm[i]][i]
        ]
        + r,
        ind if lower else reversed(ind),
        [],
    )

Verifiquemos la función con el siguiente sistema:

$$\left(\begin{matrix} 1 & 2 & 3 \\ 0 & 5 & 4 \\ 0 & 0 & 6 \end{matrix}\right) \cdot \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right) = \left(\begin{matrix} 7 \\ 8 \\ 9 \end{matrix}\right)$$

En código:

In [7]:
A = [[1,2,3],
     [0,5,4],
     [0,0,6]]
b = [7,8,9]
solvetrian(A, b)

[1.7000000000000002, 0.4, 1.5]

Si nuestra solución es correcta se debe verificar que $A \cdot x = b$. 
Veamoslo en código:

In [9]:
matmul(A, solvetrian(A, b))

array([7., 8., 9.])

#### Ejercicio 14

Usar el proceso de eliminación de Gauss Escalado para encontrar la descomposición $P \cdot A = L \cdot U$ (comparar con Python) y resolver en cada uno de los casos.

$$\left(\begin{matrix} -1 & 1 & -4 \\ 2 & 2 & 0 \\ 3 & 3 & 2 \end{matrix}\right) \cdot \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right) = \left(\begin{matrix} 0 \\ 1 \\ \frac{1}{2} \end{matrix}\right)$$

$$\left(\begin{matrix} 1 & 6 & 0 \\ 2 & 1 & 0 \\ 0 & 2 & 1 \end{matrix}\right) \cdot \left(\begin{matrix} x_1 \\ x_2 \\ x_3 \end{matrix}\right) = \left(\begin{matrix} 3 \\ 1 \\ 1 \end{matrix}\right)$$
